In [17]:
# -- Load module for singularity -- 

!module load singularity
!module load freesurfer
!module load fsl

import os 
import subprocess
import pandas as pd
from enum import Enum

from IPython.display import display, Markdown, Latex

# -- Init Variables -- 
scripts_folder='/home/traaffneu/margal/awake_code/awake/scripts/'
metadata_path ='/home/traaffneu/margal/awake_code/awake/scripts/tables/metadata_rabies.tsv'

df = pd.read_csv(metadata_path, sep='\t')
df = df.loc[(df['exclude'] != 'yes')]


### RABIES preprocessing commands

ALl scans are preprocessed one by one using defaults parameters of RABIES, execpt for the registration to the commonspace. 

**Mouse** : 
- template DSURQE (defaults template in RABIES), 
- smoothing during confound correction: 0.3

**Rat** :
- template SIGMA, 
- smoothing during confound correction: 0.5

If the QC fails, corrective parameters are applied iteratively. There are specified in the metadata_rabies file, and refered as 'correction_arg' in the present script. 

1. "--anat_autobox\ --bold_autobox"
2. "--anat_autobox --bold_autobox --bold_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false --anat_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false"
3. "--anat_autobox --bold_autobox --bold_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false --anat_inho_cor=method=N4_reg,otsu_thresh=2,multiotsu=false --bold2anat_coreg=masking=false,brain_extraction=false,registration=Rigid"



In [23]:
# --- Function to launch RABIES as a job ---

def queue_process(subj_num, specie, template_dir, input_bids_dir, output_dir, TR, template, mask, wm, csf, atlas, correction_arg, smoothing):
    
    singularity_path='/opt/singularity/3.10.3/bin/singularity'

    # -- Create the full command string --
    
    if specie == 'Rat' :
        Cmd_B_rat_template_path=f"-B {template_dir}:/template"

        rabies_cmd = f"{singularity_path} run " \
                f"{Cmd_B_rat_template_path} " \
                f"-B {input_bids_dir}:/rabies_in:ro " \
                f"-B {output_dir}:/rabies_out " \
                "/opt/rabies/0.5.0/rabies.sif -p MultiProc preprocess /rabies_in /rabies_out " \
                    f"--TR {TR} " \
                    f"--anat_template {template} " \
                    f"--brain_mask {mask} " \
                    f"--WM_mask {wm} " \
                    f"--CSF_mask {csf} " \
                    f"--vascular_mask {csf} " \
                    f"--labels {atlas} " \
                    "--commonspace_reg='masking'='false','brain_extraction'='false','template_registration'='SyN','fast_commonspace'='true' " \
                    "--commonspace_resampling '0.3x0.3x0.3' " \
                    f"{correction_arg} " \
                        
        if not os.path.exists(f"{output_dir}/confound_correction_outputs"):os.makedirs(f"{output_dir}/confound_correction_outputs")      # Create confound_correction_outputs directory
                    
    else :
        Cmd_B_rat_template_path=''

        rabies_cmd = f"{singularity_path} run " \
                    f"-B {input_bids_dir}:/rabies_in:ro " \
                    f"-B {output_dir}:/rabies_out " \
                    "/opt/rabies/0.5.0/rabies.sif -p MultiProc preprocess /rabies_in /rabies_out " \
                        f"--TR {TR} " \
                        "--commonspace_reg='masking'='false','brain_extraction'='false','template_registration'='SyN','fast_commonspace'='true' " \
                        "--commonspace_resampling '0.3x0.3x0.3' " \
                        f"{correction_arg} "
    
        if not os.path.exists(f"{output_dir}/confound_correction_outputs"):os.makedirs(f"{output_dir}/confound_correction_outputs")      # Create confound_correction_outputs directory

    confound_cmd = "/opt/singularity/3.10.3/bin/singularity run " \
                    f"-B {input_bids_dir}:/rabies_in:ro " \
                    f"-B {output_dir}:/rabies_out " \
                    f"-B {output_dir}:/preprocess_outputs/ " \
                    f"-B {output_dir}/confound_correction_outputs:/confound_correction_outputs " \
                    f"{Cmd_B_rat_template_path} " \
                    "/opt/rabies/0.5.0/rabies.sif -p MultiProc confound_correction /preprocess_outputs/ /confound_correction_outputs/ " \
                        f"--smoothing_filter {smoothing} " \
                        f"--TR {TR} " \
                        "--highpass 0.01 " \
                        "--lowpass 0.1 " \
                        "--edge_cutoff 30 " \
                                                   
    # Submit the job
    combined_cmd = f'{rabies_cmd} && {confound_cmd}'
    cmd = f'echo "Job running: $(echo \'{combined_cmd}\'| qsub -N Awk:{specie},{subj_num} -l nodes=1:ppn=1,mem=64gb,walltime=20:00:00)"'
    subprocess.call(cmd, shell=True)

In [24]:
# --- Run RABIES ---
    
for index in range(0, len(df)):
    
    # -- Define variables --
    subj_num=str(df.iloc[index]['rodent.sub'])[:-2]               #remove the 2 last characters with [:-2]
    ses_num=str(df.iloc[index]['rodent.ses'])[:-2] 
    specie = str(df.iloc[index]['rodent.spiecies']) 
    TR=str(df.iloc[index]['func.TR'])[:-2]

    if specie == 'Mouse' :
        bids_folder = '01_mice'
        template_dir=''
        template=''
        mask=''
        wm=''
        csf=''
        atlas=''
        roi=''
        smoothing = '0.3'

    else : #'Rat'
        bids_folder = '02_rats'    
        template_dir='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'
        template='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
        mask='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii'
        wm='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz'
        csf='/template/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz'
        atlas='/template/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii'
        smoothing='0.5'

    # -- Input and output directories -- 
    orig_bids_dir=f'/project/4180000.36/AwakeRodent/bids/{bids_folder}/'
    input_bids_dir=f'/project/4180000.36/AwakeRodent/scratch/bids/{bids_folder}/sub-0{subj_num}'
    output_dir=f'/project/4180000.36/AwakeRodent/scratch/RABIES_preprocess/sub-0{subj_num}_ses-{ses_num}'

    # -- Define RABIES parameters --
    class ProcessingConfig(Enum):
        DefaultParameters='0'
        AlreadyPreprocessed='1'
        CorrectiveParameters='2'
     
    rabies_preprocess_config_value=str(df.iloc[index]['rabies_prepro'])    
    rabies_preprocess_config = ProcessingConfig(rabies_preprocess_config_value)

    # -- Print scan infos -- 
    display(Markdown(f'Specie: **<span style="color:#FFA07A">{specie}</span>**   \n'
                f'Rodent num: <span style="color:#FF5733">0{subj_num}</span>  \n'     
                f'Preprocess tatus: <span style="color:#FF91A4">{rabies_preprocess_config_value}</span>  \n'
                f'Session n°: <span style="color:#45B39D">{ses_num}</span>  \n'
                f'TR: <span style="color:#0099FF">{TR}</span>  \n'))
    

    # -- Run RABIES as a job on the HPC --
    if rabies_preprocess_config == ProcessingConfig.CorrectiveParameters:
        if not os.path.exists(output_dir):os.makedirs(output_dir)      # Create directory
        correction_arg=str(df.iloc[index]['rabies_cor'])
        queue_process(subj_num, specie, template_dir, input_bids_dir, output_dir, TR, template, mask, wm, csf, atlas, correction_arg, smoothing)
        print("RABIES: corrective params.")

    elif rabies_preprocess_config == ProcessingConfig.DefaultParameters:
        if not os.path.exists(output_dir):os.makedirs(output_dir)      # Create directory
        queue_process(subj_num, specie, template_dir, input_bids_dir, output_dir, TR, template, mask, wm, csf, atlas, correction_arg='', smoothing=smoothing)
        print("RABIES: default params.") 
        print('-----------')    


Specie: **<span style="color:#FFA07A">Mouse</span>**   
Rodent num: <span style="color:#FF5733">0100100</span>  
Preprocess tatus: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1</span>  


Job running: 50425816.dccn-l029.dccn.nl
RABIES: default params.
-----------


Specie: **<span style="color:#FFA07A">Rat</span>**   
Rodent num: <span style="color:#FF5733">0200100</span>  
Preprocess tatus: <span style="color:#FF91A4">0</span>  
Session n°: <span style="color:#45B39D">1</span>  
TR: <span style="color:#0099FF">1</span>  


Job running: 50425817.dccn-l029.dccn.nl
RABIES: default params.
-----------


In [16]:
# --- Run only CONFOUNDS ---

subj_num='200100'             #remove the 2 last characters with [:-2]
ses_num='1'
specie = 'Rat'   
bids_folder = '02_rats'   
print(subj_num)
TR='1.0'
smoothing = 0.5

# -- Templates --
template_dir='/groupshare/traaffneu/preclinimg/templates/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1'
# template='/templates/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii'
# mask='/templates/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii'
# wm='/templates/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz'
# csf='/templates/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz'
# atlas='/templates/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii'
# roi='/templates/groupshare/traaffneu/preclinimg/template/roi/'


# -- Input and output directories -- 
orig_bids_dir=f'/project/4180000.36/AwakeRodent/bids/{bids_folder}/'
input_bids_dir=f'/project/4180000.36/AwakeRodent/scratch/bids/{bids_folder}/sub-0{subj_num}'
output_dir=f'/project/4180000.36/AwakeRodent/scratch/RABIES_preprocess/sub-0{subj_num}_ses-{ses_num}'

output_dir=f'/project/4180000.36/AwakeRodent/scratch/RABIES_preprocess/sub-0{subj_num}_ses-{ses_num}'
if not os.path.exists(f"{output_dir}/confound_correction_outputs"):os.makedirs(f"{output_dir}/confound_correction_outputs")      # Make directories

Cmd_AddRatTemplatePath=f"-B {template_dir}:/template " \
#Cmd_AddRatTemplatePath=''

confound_cmd = "/opt/singularity/3.10.3/bin/singularity run " \
                    f"-B {input_bids_dir}:/rabies_in:ro " \
                    f"-B {output_dir}:/rabies_out " \
                    f"-B {output_dir}:/preprocess_outputs/ " \
                    f"-B {output_dir}/confound_correction_outputs:/confound_correction_outputs " \
                    f"{Cmd_AddRatTemplatePath}" \
                    "/opt/rabies/0.5.0/rabies.sif -p MultiProc confound_correction /preprocess_outputs/ /confound_correction_outputs/ " \
                    f"--smoothing_filter {smoothing} " \
                    f"--TR {TR} " \
                    "--highpass 0.01 " \
                    "--lowpass 0.1 " \
                    "--edge_cutoff 30 " \

                    #f"-B {template_dir}:/template " \
                        
# cmd = f'echo "{confound_cmd}" | qsub -N Cfond:{specie},{subj_num} -l nodes=1:ppn=1,mem=64gb,walltime=4:00:00'
# subprocess.call(cmd, shell=True)

cmd = f'echo "Job running: $(echo \'{confound_cmd}\' | qsub -N Cfond:{specie},{subj_num} -l nodes=1:ppn=1,mem=64gb,walltime=4:00:00)"'
subprocess.call(cmd, shell=True)


                       

200100
Job running: 50425812.dccn-l029.dccn.nl


0